READ PERMIT DATA

In [1]:
from datetime import datetime
print str(datetime.now())

import pandas as pd
#import logging
pd.set_option('chained_assignment', None)

from datetime import datetime
print str(datetime.now())

#logging.basicConfig(filename="I:\\BAT\\SITE SAFETY AUTOMATION\\automation_log.log",level=logging.DEBUG)
#logging.debug('This message should go to the log file')
#logging.warning('And this, too')

#text_file = open("I:\\BAT\\SITE SAFETY AUTOMATION\\automation_log.txt", "a" )


#string = str(datetime.now()) + "..." + "Reading active permit data"  + "\n"
#print string
#text_file.write(" \n")
#text_file.write("******\n")
#text_file.write(string)





def PermitDataInput():
    #import datetime
    #import time
    #from datetime import date
    #from dateutil.relativedelta import relativedelta
    #from dateutil import parser

    #permits issued
    df = pd.read_csv('I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\active permitsAuto.csv', low_memory=False)
    
    
    
    
    #drop columns
    df = df.drop('Borough Name', 1)
    df = df.drop('Current Job Status', 1)
    df = df.drop('Permit Filing Status (Raw)', 1)
    df = df.drop('Permit Sequence Number Max Flag', 1)
    df = df.drop('issue_DATE', 1)
    df = df.drop('issue_YEAR', 1)
    df = df.drop('Permit Filing Status', 1)
    df = df.drop('Job-Permit', 1)
    
    #df = df.drop('Applicant Business Name', 1)
    
    dropList =   ["Applicant First Name", "Applicant Last Name", "Applicant License Number", 
                  "Applicant License Type", "Applicant Middle Initial"]   

    df = df.drop(dropList, 1)
    
    
  


    print "length before drop", len(df)

    before = len(df)

    df = df.dropna(subset = ['expire_DATE'])

    print "length after drop", len(df)

    print "number dropped", before - len(df)

    df = df.reset_index(drop=True)
    
    #ts = time.time()
    #st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    #print "Current Date", st
  
    #df['expire_DATE'] = pd.to_datetime(df['expire_DATE'])
    #df = df.sort_values(by = 'expire_DATE', ascending=True).reset_index(drop=True)
    
    #df = df[(df['expire_DATE'] >= st) ]
    
    df = df.rename(columns={'Latitude Point': 'Latitude', 'Longitude Point': 'Longitude', 'Borough Name.1': 'Borough',
                           'Applicant Business Name':'General Contractor'})

    df = df.reindex_axis(sorted(df.columns), axis=1)
    
    
    return df


#print "Reading BIS job filing data ..."
#df = PermitDataInput()

#print df.dtypes


#IMPORT DOB NOW DATA
def DNDataInput():
    df = pd.read_csv('I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\Active Permits - DOB NOWAuto.csv', low_memory=False)
    
    df["Permit Type"] = "EQ"
    df["Proposed Stories"] = ""
    df = df.rename(columns={'Work Type Name': 'Permit Sub Type', 'ExisitingBuildingStories':'Existing Stories',
                           'Permit Expiration Date':'expire_DATE', 'JobType':'Job Type', 
                            'Business Name':'General Contractor'})
    
    
    
    df = df.drop('Sequence Number', 1)
    df = df.drop('Permit Issued Date', 1)
    df = df.drop('Filing Type', 1)
    df = df.drop('Filing Status', 1)
    
    df = df.drop('Last Name', 1)
    df = df.drop('First Name', 1)
    df = df.drop('MiddleI nitial', 1)
    #df = df.drop('Business Name', 1)
    
    df = df.drop('License', 1)
    df = df.drop('License Number', 1)
    
    df['Permit Sub Type'] = df['Permit Sub Type'].str.replace('Side Walk Shed', 'SH')
    df['Permit Sub Type'] = df['Permit Sub Type'].str.replace('Scaffold', 'SF')
    df['Permit Sub Type'] = df['Permit Sub Type'].str.replace('Fence', 'FN')
    
    df = df.rename(columns={'Bin': 'BIN Number'})
    df = df.reindex_axis(sorted(df.columns), axis=1)
    
    print "len of dob now data", len(df)

    
    return df

#dobnow = DNDataInput()
#print dobnow.dtypes


def CODataInput():
    df = pd.read_csv('I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\C of O Issued (BIS)Auto.csv', low_memory=False)
    df = df[["J_BIN_NUMBER", "J_JOB_NUMBER", "AP_A_ISSUE_TYPE", "C of O Issue Date"]]
    df["J_BIN_NUMBER"] = df["J_BIN_NUMBER"].astype(str)
    df["J_BIN_NUMBER"] = df["J_BIN_NUMBER"].map(str.strip)
    df = df.rename(columns={'J_BIN_NUMBER': 'BIN Number', 'AP_A_ISSUE_TYPE': 'CO_TCO', 'J_JOB_NUMBER':'Job Number'})
    
    return df
    
#CREATE DATAFRAMES
print "Reading BIS job filing data ..."
#text_file.write(str(datetime.now()) + "..." + "Reading BIS job filing data"  + "\n")
df = PermitDataInput()
print "Reading DOB Now job filing data ..."
#text_file.write(str(datetime.now()) + "..." + "Reading DOB Now job filing data"  + "\n")
dobnow = DNDataInput()
print "Reading CO/TCO data for dictionary ..."
#text_file.write(str(datetime.now()) + "..." + "Reading CO/TCO data for dictionary"  + "\n")
co = CODataInput()



df["BIN Number"] = df["BIN Number"].astype(str)
df["BIN Number"] = df["BIN Number"].map(str.strip)
dobnow["BIN Number"] = dobnow["BIN Number"].astype(str)
dobnow["BIN Number"] = dobnow["BIN Number"].map(str.strip)

#INSERT THE CORRECT LAT LON COORDINATES FROM BIS INTO DOBNOW
latDic = df.set_index('BIN Number')['Latitude'].to_dict()
lonDic = df.set_index('BIN Number')['Latitude'].to_dict()

for i in range(0, len(dobnow)):
    if dobnow["BIN Number"][i] in latDic:
        dobnow["Latitude"][i] = latDic[dobnow["BIN Number"][i]]
    if dobnow["BIN Number"][i] in lonDic:
        dobnow["Longitude"][i] = lonDic[dobnow["BIN Number"][i]]


frames = [df, dobnow]
result = pd.concat(frames)

df = result.reset_index(drop=True)
df['expire_DATE'] = pd.to_datetime(df['expire_DATE'])
df = df.sort_values(by = 'BIN Number', ascending=True).reset_index(drop=True)

df["Borough"] = df["Borough"].astype(str)
df["Borough"] = df["Borough"].map(str.upper)


print "lengh of combined", len(df)



2018-06-25 09:56:07.835000
2018-06-25 09:56:32.126000
Reading BIS job filing data ...
length before drop 96782
length after drop 96782
number dropped 0
Reading DOB Now job filing data ...
len of dob now data 4324
Reading CO/TCO data for dictionary ...
lengh of combined 101106


AGGREGATE AND COUNT DATA

In [2]:
def GetStories(dtf):    
    existStories = dtf[dtf["Existing Stories"] != 0]
    existStories = existStories["Existing Stories"].tolist()
    existStories = filter(None, existStories)
    #existStories = sorted(existStories, reverse=True)
    #print existStories
    
    proposedStories = dtf[dtf["Proposed Stories"] != 0]
    proposedStories = proposedStories["Proposed Stories"].tolist()
    proposedStories = filter(None, proposedStories)
    #proposedStories = sorted(proposedStories, reverse=True)
    #print proposedStories

    #stories
    if len(proposedStories) > 0:
        #if len(existStories) != 0:
        stories = proposedStories[0]
    elif len(existStories) > 0:
        stories = existStories[0]
    else:
        stories = 0
    return stories



print "Aggregating to BIN level ..."
#text_file.write(str(datetime.now()) + "..." + "Aggregating to BIN level"  + "\n")


qn = pd.DataFrame(columns=('BIN', 'Address', 'Borough', 'Community Board', 'Number Permits' 
                           'Number Stories', 'NB', 'ShedScafFence', 'Job Type List'  'Latitude', 'Longitude'))

binList = sorted(list(set(df["BIN Number"].tolist())))

print "Number of BINs", len(binList)


AggList = []

df["Job Type"] = df["Job Type"].astype(str)
df["Job Type"] = df["Job Type"].map(str.strip)

df["Job Number"] = df["Job Number"].astype(str)
df["Job Number"] = df["Job Number"].map(str.strip)

df["Permit Type"] = df["Permit Type"].astype(str)
df["Permit Type"] = df["Permit Type"].map(str.strip)

df["Permit Sub Type"] = df["Permit Sub Type"].astype(str)
df["Permit Sub Type"] = df["Permit Sub Type"].map(str.strip)

for i in range(0, len(binList)):
#for i in range(0, 5):    
    if i%(len(binList)/10) == 0:
        print i
    
    df2 = df[df["BIN Number"] == binList[i]]
    df2 = df2.reset_index(drop=True)
    
    df2 = df2.sort_values(by = 'expire_DATE', ascending=False).reset_index(drop=True)
    
    #print df2
    
    #COUNT NUMBER OF PERMITS
    df2["Permit Agg"] = df2["Job Type"] + " " + df2["Permit Type"] + " " + df2["Permit Sub Type"]
   
    PermitAggList = list(set(df2["Permit Agg"].tolist() ) )
    if 'nan' in PermitAggList:
        PermitAggList.remove('nan')
    
    numPermits = len(PermitAggList)
    if len(PermitAggList) == 0:
        print PermitAggList, binList[i]
    
    #print df2
    #print
    #print PermitAggList


    
    
    
    #DETERMIN NUMBER OF FLOORS, LOOK AT NB FIRST, THEN DM, THEN AFTER THE MOST RECENT
    storiesNB = df2.copy()
    storiesNB = storiesNB[storiesNB["Job Type"] == "NB"]
    storiesDM = df2.copy()
    storiesDM = storiesDM[storiesDM["Job Type"] == "DM"]
    storiesA1 = df2.copy()
    storiesA1 = storiesA1[storiesA1["Job Type"] == "A1"]
    
    if len(storiesNB) > 0:
        stories = GetStories(storiesNB)
    elif len(storiesDM) > 0:
        stories = GetStories(storiesDM) 
    elif len(storiesA1) > 0:
        stories = GetStories(storiesA1)
    else:
        a2 = df2.copy()
        stories = GetStories(a2)
           
        
    #NB
    #job_types = df2["Job Type"].tolist()
    #if "NB" in job_types:
    #    nb = "YES"
    #else:
    #    nb = "NO"
        
    #shed, scaffold or fence
    permitSub = df2["Permit Sub Type"].tolist()
    if ('SH' in permitSub or 'SF' in permitSub or 'FN' in permitSub):
        ssf = "YES"
    else:
        ssf = "NO"
        
        
    #JOB TYPE AND JOB NUMBER, MAJOR JOB TYPES ONLY
    jobTypeList = ["NB", "DM", "A1"]
    df3 = df2.copy()
    df3 = df3[df3['Job Type'].isin(jobTypeList)]
    
    #df3 = df2.copy()
    jt_number = []
    contList = []
    if len(df3) > 0:
        df3["Job Type Number"] = df3["Job Type"] + " " + df3["Job Number"]
        jt_number = df3["Job Type Number"].tolist()
        contList = df3["General Contractor"].tolist()
        
        
    #GET JOB TYPE, THE HIGHEST IN HIEARCHY,JOB NUMBER AND CONTRACTOR
    nb = df2.copy()
    nb = nb[nb["Job Type"] == "NB"]
    nb = nb.reset_index(drop=True)
    dm = df2.copy()
    dm = dm[dm["Job Type"] == "DM"]
    dm = dm.reset_index(drop=True)
    a1 = df2.copy()
    a1 = a1[a1["Job Type"] == "A1"]
    a1 = a1.reset_index(drop=True)
    a2 = df2.copy()
    a2 = a2[a2["Job Type"] == "A2"]
    a2 = a2.reset_index(drop=True)
    a3 = df2.copy()
    a3 = a3[a3["Job Type"] == "A3"]
    a3 = a3.reset_index(drop=True)

    
    if len(nb) > 0:
        jobtype = nb["Job Type"][0]
        jobnumber = nb["Job Number"][0]
        contractor = nb["General Contractor"][0]
    elif len(dm) > 0:
        jobtype = dm["Job Type"][0]
        jobnumber = dm["Job Number"][0]
        contractor = dm["General Contractor"][0]
    elif len(a1) > 0:
        jobtype = a1["Job Type"][0]
        jobnumber = a1["Job Number"][0]
        contractor = a1["General Contractor"][0]
    elif len(a2) > 0:
        jobtype = a2["Job Type"][0]
        jobnumber = a2["Job Number"][0]
        contractor = a2["General Contractor"][0]
    else:
        jobtype = a3["Job Type"][0]
        jobnumber = a3["Job Number"][0]
        contractor = a3["General Contractor"][0]
          
     
    
    #print "***************"
    #print df3
    
    
    
    
    
    AggList.append([binList[i],df2["Address"][0], df2["Borough"][0], df2["Community Board"][0], 
                    numPermits, stories, ssf,jt_number,jobtype, jobnumber, contractor, contList, df2["Latitude"][0], df2["Longitude"][0] ] )
    #print
    #print df2
    
    
data = pd.DataFrame(AggList)

data = data.rename(columns={0: 'BIN', 1: 'Address', 2:'Borough', 3:'Community Board', 4:'Number of Permits',
                            5:'Number of Stories',  6:'ScaffoldShedFence', 7:'Job Type and Number List', 8:'Job Type',
                            9:'Job Number', 10:'Contractor', 11:'Contractor List',
                            12:'Lat', 13:'Lon'})



Aggregating to BIN level ...
Number of BINs 39909
0
3990
7980
11970
15960
19950
23940
27930
31920
35910
39900


IMPORT ACCIDENT DATA AND INSERT INTO DATA

In [3]:
pd.set_option('chained_assignment', None)

#text_file.write(str(datetime.now()) + "..." + "Import accident data and insert into dataframe"  + "\n")


def AccidentDataInput():

    #import datetime
    #import time
    #from datetime import date
    #from dateutil.relativedelta import relativedelta
    #from dateutil import parser

    #permits issued
    df = pd.read_csv('I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\construction related accidents_3 YRS PERIODAuto.csv', low_memory=False)
    
    df = df[["BIN"]]
    
    print "length before drop nan", len(df)
    df = df.dropna()
    print "length after drop nan", len(df)
    
    df["BIN"] = df["BIN"].astype(int)
    df["BIN"] = df["BIN"].astype(str)
    df["BIN"] = df["BIN"].map(str.strip)



    
    df["Value"] = 1
    
    df = df.groupby(['BIN']).sum()
    df = df.add_suffix('').reset_index()
    
    df = df.set_index('BIN')['Value'].to_dict()
    
    return df

AccDic = AccidentDataInput()

#INSERT ACCIDENTS
data["Accidents"] = 0
data["BIN"] = data["BIN"].astype(str)
data["BIN"] = data["BIN"].map(str.strip)

for i in range(0, len(data)):
    if data["BIN"][i] in AccDic:
        data["Accidents"][i] = AccDic[data["BIN"][i]]
        
#data = data[["BIN", "Number of Permits", "Number of Stories", "NB", "ScaffoldShedFence", "Accidents", "Lat", "Lon"]]

#data = data.reindex_axis(sorted(df.columns), axis=1)

length before drop nan 1869
length after drop nan 1838


INSERT CO/TCO DATA

In [4]:
#keep only active job numbers

#text_file.write(str(datetime.now()) + "..." + "Insert CO and TCO data"  + "\n")


count = 0

df["Job Number"] = df["Job Number"].astype(str)
co["Job Number"] = co["Job Number"].astype(str)

dfJobNum = list(set(df["Job Number"].tolist() ) )

print "Remove those CO/TCO not active ..."
print "Length of co", len(co)

co = co[co['Job Number'].isin(dfJobNum)]
co = co.reset_index(drop=True)

print "Length after drop co", len(co)


print "Creating CO, TCO dictionary ..."
#includes co and tco
data["CO/TCO"] = "NO"
data["CO/TCO Recent Date"] = ""

coDic = co.set_index('BIN Number')['CO_TCO'].to_dict()


#create dictionary

co_binList = list( set(co["BIN Number"].tolist() ) )

co['C of O Issue Date'] = pd.to_datetime(co['C of O Issue Date'])

dateDic = {}

print "Creating date dictionary ..."

coG = co[["BIN Number", "C of O Issue Date"]]
#get the most recent date on the BIN
coG = coG.groupby(['BIN Number'], sort=False)['C of O Issue Date'].max()

#convert series to dataframe
coG_df = pd.DataFrame(coG)

coG_df = coG_df.add_suffix('').reset_index()

coG_df["C of O Issue Date"] = coG_df["C of O Issue Date"].astype(str)

coG_df


dateDic = coG_df.set_index('BIN Number')['C of O Issue Date'].to_dict()




print "Inserting certificat of occupancy and recent date ... "
for i in range(0, len(data)):
    if data["BIN"][i] in coDic:
        data["CO/TCO"][i] = "YES" 
    
        data["CO/TCO Recent Date"][i] = dateDic[data["BIN"][i]]
        
        count+=1
            
print "Number that match", count
        
        



Remove those CO/TCO not active ...
Length of co 39351
Length after drop co 2853
Creating CO, TCO dictionary ...
Creating date dictionary ...
Inserting certificat of occupancy and recent date ... 
Number that match 655


CREATE THE SCORES

In [5]:
#text_file.write(str(datetime.now()) + "..." + "Score the data"  + "\n")
print "Creating the scores..."

data['Score'] = 0
for i in range(0, len(data)):
    score = []
    #number of permits
    if (data["Number of Permits"][i] >= 1 and data["Number of Permits"][i] <= 4):
        score.append(1)
    if (data["Number of Permits"][i] >= 5 and data["Number of Permits"][i] <= 7):
        score.append(2)
    if (data["Number of Permits"][i] >= 8 and data["Number of Permits"][i] <= 10):
        score.append(3)
    if (data["Number of Permits"][i] >= 11 ):
        score.append(4)
    
    if  "NB" in data["Job Type"][i] > 0:
        score.append(1)
    else:
        score.append(0)
        
    if data["ScaffoldShedFence"][i] > 0:
        score.append(1)
    else:
        score.append(0)
        
    if data["Number of Stories"][i] >= 10:
        score.append(1)
    else:
        score.append(0)
        
    if data["Accidents"][i] > 0:
        score.append(1)
    else:
        score.append(0)
        
        
    data['Score'][i] = sum(score)
        


Creating the scores...


OUTPUT ALL DATA

In [6]:
#data.to_csv("I:\\BAT\\SITE SAFETY AUTOMATION\\test.csv", index=False)
data.dtypes

BIN                          object
Address                      object
Borough                      object
Community Board             float64
Number of Permits             int64
Number of Stories             int64
ScaffoldShedFence            object
Job Type and Number List     object
Job Type                     object
Job Number                   object
Contractor                   object
Contractor List              object
Lat                         float64
Lon                         float64
Accidents                     int64
CO/TCO                       object
CO/TCO Recent Date           object
Score                         int64
dtype: object

OUTPUT SUBSETTED DATA

In [7]:
output = data.copy()

#output["Job Type and Number List"] = output["Job Type and Number List"].astype(str)
#output["Job Type and Number List"] = output["Job Type and Number List"].str.replace("'","")
#output["Job Type and Number List"] = output["Job Type and Number List"].str.replace("[", '')
#output["Job Type and Number List"] = output["Job Type and Number List"].str.replace("]", '')

#output["Contractor List"] = output["Contractor List"].astype(str)
#output["Contractor List"] = output["Contractor List"].str.replace("'","")
#output["Contractor List"] = output["Contractor List"].str.replace("[", '')
#output["Contractor List"] = output["Contractor List"].str.replace("]", '')

#drop columns
output = output.drop('Job Type and Number List', 1)
output = output.drop('Contractor List', 1)

output["Contractor"] = output["Contractor"].str.replace(',', '')
output["Address"] = output["Address"].str.replace(',', '')

#output["Job Type and Number"] = output["Job Type and Number"].str.replace('[^\w\s]','')

#output["Job Type and Number List"] = output["Job Type and Number List"].map(str.strip)

#remove low risk
print "length before drop low risk", len(output)
output = output[output["Score"] > 4]
output = output.reset_index(drop=True)
print "len after drop", len(output)

#remove low risk
print "drop CO/TCO"
output = output[output["CO/TCO"] != "YES"]
output = output.reset_index(drop=True)
print "len after drop", len(output)



length before drop low risk 39909
len after drop 528
drop CO/TCO
len after drop 419


In [8]:
#EXPORT AS CSV AND JSON FILE
#text_file.write(str(datetime.now()) + "..." + "Output the data"  + "\n")
output.to_csv("I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\SiteSafetyAutomationOutput.csv", index=False)
#text_file.write(str(datetime.now()) + "..." + "DONE"  + "\n")
#text_file.close()

#EXPORT JSON
import csv
import json
reader = csv.DictReader(open('I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\SiteSafetyAutomationOutput.csv', 'rb'))

dict_list = []
for line in reader:
    dict_list.append(line)

    
with open('I:\\BAT\\SITE SAFETY AUTOMATION\\sourceData\\SiteSafetyAutomationOutput.json', 'w') as outfile:
    json.dump(dict_list, outfile)

print str(datetime.now())

print "DONE"

2018-06-25 10:09:15.302000
DONE
